In [ ]:
import torchaudio
import torch
import os

import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset

device = torch.device("cuda")

## Load Data

In [ ]:
class AnimeAudioDataset(Dataset):
    """
    Load in audio file names and get the length of the largest audio sample.
    Audio is loaded into memory when _get_item is called
    
    Labels are all loaded into memory and normalized during initialization
    """
    
    def __init__(self):

        self.audio_dir = 'data/Audio'
        self.label_dir = 'data/Labels/nick'
        
        self.audio_files, self.max_length = self._load_audio()
        raw_labels = self._load_labels(self.audio_files)
        self.labels, self.l_mean, self.l_std = self._normalize_labels(raw_labels)
        self.labels = self.labels.to(device)
    
        assert len(self.audio_files) == len(self.labels)
        
        '''
        self.data = self._pad_audio(self._load_audio())
        self.labels, self.label_mean, self.label_std = \
                self._normalize_labels(self._load_labels())
        '''
        
    def _load_audio(self):

        audio_files = []
        max_length = 0
        
        for a in os.listdir(self.audio_dir):
            audio_files.append(a)
            
            audio_file = os.path.join(self.audio_dir, a)
            waveform, _ = torchaudio.load(audio_file)
            max_length = max(max_length, waveform.shape[1])
        
        return audio_files, max_length
    
    def _load_labels(self, audio_files):

        labels = []
        error  = False
        
        for filename in audio_files:
            file = os.path.splitext(filename)[0] + '.label'
            filepath = os.path.join(self.label_dir, file)
            label = [0] * 4
            try:
                f = open(filepath, 'r')
                for i in range(4):
                    label[i] = int(f.readline())
                labels.append(label)
            except BaseException as e:
                print(e)
                error = True
        
        if error:
            raise RuntimeError('Failed to load labels')
            
        return torch.Tensor(labels)
                
    def _normalize_labels(self, labels):
        l_mean = labels.mean(dim=0)
        l_std = labels.std(dim=0)
        labels = (labels - l_mean) / l_std
        return labels, l_mean, l_std
        
    def _pad_audio(self, data):
        # longest = max(map(lambda x: x.shape[1], data))
        for i in range(len(data)):
            zeros = torch.zeros(2, (self.max_length - data[i].shape[1]))
            data[i] = torch.cat((data[i], zeros), dim=1)
        return torch.stack(data)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # idx can be a tensor
        audio_file = os.path.join(self.audio_dir, self.audio_files[idx])
        waveform, _ = torchaudio.load(audio_file)
        
        padded_audio = self._pad_audio([waveform])
        padded_audio = padded_audio.to(device)
        return padded_audio, self.labels[idx]

anime_audio_data = AnimeAudioDataset() 

## Define Network

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.magic_num = 1103908 #234395 * 4
        self.conv1 = nn.Conv1d(2, 4, 1600, stride=10)
        self.pool = nn.MaxPool1d(5)
        # self.conv2 = nn.Conv1d(4, 8, 400, stride=10)
        self.fc1 = nn.Linear(self.magic_num, 4)
        # self.fc2 = nn.Linear(10000, 100)
        # self.fc3 = nn.Linear(100, 4)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.magic_num)
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        x = self.fc1(x)
        return x

In [ ]:
net = Net()
net.cuda()
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.003, momentum=0)

# Train

In [ ]:
for epoch in range(10):
    
    running_loss = 0
    
    for data, label in anime_audio_data:
    
        optimizer.zero_grad()
        data, label = data, label.unsqueeze(0)
        
        output = net(data)
        loss = criterion(output, label)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        
    print("loss:", running_loss)

# Compare actual labels

In [ ]:
label_1 = "data/Labels/nick/One_Punch_Man_1.label"
label_2 = "data/Labels/nick/One_Punch_Man_5.label"
label_3 = "data/Labels/nick/One_Punch_Man_6.label"

labels = []
for filename in [label_1, label_2, label_3]:
    f = open(filename, "r")
    label = []
    for i in range(4):
        label.append(int(f.readline()))
    labels.append(label)

labels = torch.Tensor(labels)
labels = labels.to(device)

In [ ]:
for i, data in enumerate(anime_audio_data):
    data, _ = data
    data = data.unsqueeze(0)
    out = net(data) * anime_audio_data.label_std.to(device) + anime_audio_data.label_mean.to(device)
    print(out)
    print(labels[i])

In [ ]:
# util to find magic numbers
x = data[0].unsqueeze(0)
x = net.pool(F.relu(net.conv1(x)))
print(x.shape)
x = x.view(-1, 5859 * 16)
# print(x.shape)
x = F.relu(net.fc1(x))
x = F.relu(net.fc2(x))
x = net.fc3(x)
print(x)
pass

In [ ]:
# find magic num
x = data[0].unsqueeze(0)
x = net.pool(F.relu(net.conv1(x)))
# x = net.pool(F.relu(net.conv2(x)))
print(x.shape)
x = x.view(-1, net.magic_num)
x = net.fc1(x)
print(x)